In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from vaegs.model import *
from vaegs.utils import *
from vaegs.train import *
import warnings
warnings.filterwarnings("ignore")

data599 = np.loadtxt('./data/wheat599_X.pkl.csv', delimiter=',')
data599 = data599[:,1:]
type_1 = np.loadtxt('./data/wheat1.Y', delimiter=',',skiprows=1)
type_2 = np.loadtxt('./data/wheat2.Y', delimiter=',',skiprows=1)
type_3 = np.loadtxt('./data/wheat3.Y', delimiter=',',skiprows=1)
type_4 = np.loadtxt('./data/wheat4.Y', delimiter=',',skiprows=1)
type599 = np.stack((type_1,type_2,type_3,type_4)).T
print(data599.shape,type599.shape)

(599, 1279) (599, 4)


In [ ]:
device = try_gpu(2)
out_dim = type599.shape[1]
file_name='wheat599.csv'
net = VMGP(data599.shape[1],out_dim=out_dim)
batch_size = 32
train_test_klcv(net,data599, type599,num_epochs=50,lr=0.0001, print_skip = 50, device = device, 
                batch_size=batch_size, file_name=file_name)

In [ ]:
def pre_train_kl(net, train_data, device, file_name, num_epochs=10, lr=1e-5, batch_size=32, shuffle=True, num_workers=0, print_skip = 50, opt = 'adam'):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('pre training on', device)
    net.to(device)
    if opt == 'adam':
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    elif opt == 'sgd':
        optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.MSELoss()
    # 训练vae
    for epoch in range(num_epochs):
        # print('epoch:', epoch)
        net.train()
        el = 0
        train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        for batch_idx, (X, _) in enumerate(train_loader):
            optimizer.zero_grad()
            X = X.to(device)
            out = net(X)
            X_hat = out[1]
            mu, logvar = out[2],out[3]
            kl_loss = -0.5*torch.mean(logvar+1-mu**2-torch.exp(logvar))   
            l = loss(X_hat, X) + kl_loss
            el += l
            l.backward()
            optimizer.step()
        if epoch==0 or (epoch+1) % print_skip ==0:
            print('epoch %d loss:'%(epoch+1), el/(batch_idx+1))
    save_model(net, 'saved_models/'+file_name)

In [ ]:
device = try_gpu(2)
out_dim = type2.shape[1]
net = VaeMultiNet3(data2.shape[1],  out_dim=out_dim)
all_data = MyDataSet(data2, type2)
pre_train_kl(net, all_data, device, 'wheat599_kl_pre.h5', num_epochs=500, lr=0.0001, batch_size=32)

试验设计有问题，没有每个fold重新加载模型，效果巨好是因为预测数据模型已经见过了

In [ ]:
device = try_gpu(2)
out_dim = type2.shape[1]
samples = [50,None]
file_name='wheat599_kl.csv'
# appendresult(['rsample-m'], file_name)
for sample in samples:
    net = load_model('saved_models/wheat599_kl_pre.h5', device)
    batch_size = 32
    print('sample ',sample, 'with ', batch_size)
    train_test_klcv(net,data2, type2,num_epochs=50,lr=0.0001, print_skip = 50, device = device, batch_size=batch_size,
                  sample_size=sample, file_name=file_name,init=False)

In [ ]:
device = try_gpu(2)
out_dim = type2.shape[1]
samples = [50,None]
file_name='wheat599_kl.csv'
# appendresult(['rsample-m'], file_name)
for sample in samples:
    net = load_model('saved_models/wheat599_kl_pre.h5', device)
    batch_size = 32
    print('sample ',sample, 'with ', batch_size)
    train_test_klcv(net,data2, type2,num_epochs=50,lr=0.0001, print_skip = 50, device = device, batch_size=batch_size,
                  sample_size=sample, file_name=file_name,init=False, net_name = 'wheat599_kl_pre.h5')

In [ ]:
device = try_gpu(2)
out_dim = type2.shape[1]
samples = [50,None]
file_name='wheat599_kl.csv'
# appendresult(['rsample-m'], file_name)
for sample in samples:
    net = load_model('saved_models/wheat599_kl_pre.h5', device)
    batch_size = 32
    print('sample ',sample, 'with ', batch_size)
    train_test_klcv(net,data2, type2,num_epochs=10,lr=0.0001, print_skip = 50, device = device, batch_size=batch_size,
                  sample_size=sample, file_name=file_name,init=False, net_name = 'wheat599_kl_pre.h5')

In [ ]:
device = try_gpu(2)
out_dim = type2.shape[1]
samples = [50,None]
file_name='wheat599_kl.csv'
# appendresult(['rsample-m'], file_name)
for sample in samples:
    net = load_model('saved_models/wheat599_kl_pre.h5', device)
    batch_size = 32
    print('sample ',sample, 'with ', batch_size)
    train_test_klcv(net,data2, type2,num_epochs=100,lr=0.0001, print_skip = 50, device = device, batch_size=batch_size,
                  sample_size=sample, file_name=file_name,init=False, net_name = 'wheat599_kl_pre.h5')